In [54]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

In [55]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))])

trainset = torchvision.datasets.MNIST(root='./data',
                                     train=True,
                                     download=False,
                                     transform=transform)

trainloader = torch.utils.data.DataLoader(trainset,
                                         batch_size=100,
                                         shuffle=True,
                                         num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', 
                                        train=False, 
                                        download=False, 
                                        transform=transform)

testloader = torch.utils.data.DataLoader(testset,
                                        batch_size=100,
                                        shuffle=False, 
                                        num_workers=2)

In [56]:
import torch.nn as nn
import torch.nn.functional as F

In [58]:
params = list(net.parameters())
for param in params:
    print(param.size())

torch.Size([32, 1, 3, 3])
torch.Size([32])
torch.Size([32, 32, 3, 3])
torch.Size([32])
torch.Size([64, 32, 3, 3])
torch.Size([64])
torch.Size([64, 64, 3, 3])
torch.Size([64])
torch.Size([128, 1024])
torch.Size([128])
torch.Size([10, 128])
torch.Size([10])


In [148]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.conv4 = nn.Conv2d(64, 64, 3)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(4 * 4 * 64, 128)
        self.dropout1 = nn.Dropout2d(p=0.1)
        self.fc2 = nn.Linear(128, 10)
        self.dropout2 = nn.Dropout2d(p=0.1)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.pool2(F.relu(self.conv4(x)))
#         x = F.relu(self.conv5(x))
#         x = self.pool3(F.relu(self.conv6(x)))
        x = x.view(-1, 4 * 4 * 64)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.dropout2(self.fc2(x))
        x = F.log_softmax(x)
        return x

net = Net()

In [149]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [150]:
epochs = 3

for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print('[{:d}, {:5d}] loss: {:.3f}'
                    .format(epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


[1,   100] loss: 0.678
[1,   200] loss: 0.160
[1,   300] loss: 0.118
[1,   400] loss: 0.101
[1,   500] loss: 0.076
[1,   600] loss: 0.078
[2,   100] loss: 0.052
[2,   200] loss: 0.061
[2,   300] loss: 0.057
[2,   400] loss: 0.049
[2,   500] loss: 0.054
[2,   600] loss: 0.050
[3,   100] loss: 0.036
[3,   200] loss: 0.038
[3,   300] loss: 0.034
[3,   400] loss: 0.037
[3,   500] loss: 0.037
[3,   600] loss: 0.035
Finished Training


In [152]:
correct = 0
total = 0

with torch.no_grad():
    for (images, labels) in testloader:
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy: {:.2f} %%'.format(100 * float(correct/total)))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy: 99.10 %%
